In [1]:

import os
import pandas as pd
import json

Extracting Data from JSON filre to Data Frame in Pandas

In [5]:
import os, json
import pandas as pd
from pathlib import Path

# ---- set your base path (ABSOLUTE) ----
BASE = Path("/Users/kishore_kumar/PhonePe/pulse/data/aggregated/transaction/country/india/state")

if not BASE.exists():
    raise FileNotFoundError(f"Base path not found: {BASE}")

rows = {
    'State': [], 'Year': [], 'Quarter': [],
    'Transaction_type': [], 'Transaction_count': [], 'Transaction_amount': []
}

# walk: state -> year -> quarter.json
for state_dir in sorted([d for d in BASE.iterdir() if d.is_dir()]):
    state = state_dir.name
    for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()]):
        year = int(year_dir.name)
        for qfile in sorted([f for f in year_dir.iterdir() if f.suffix == ".json"], key=lambda p: int(p.stem)):
            try:
                with qfile.open("r", encoding="utf-8") as f:
                    js = json.load(f)
            except Exception as e:
                print(f"[WARN] skip {qfile}: {e}")
                continue

            txn_list = (js or {}).get("data", {}).get("transactionData", []) or []
            for item in txn_list:
                name = item.get("name")
                instruments = item.get("paymentInstruments", []) or []

                # Prefer TOTAL; if missing, sum all instruments
                total_count, total_amount = None, None
                for inst in instruments:
                    if inst.get("type") == "TOTAL":
                        total_count = inst.get("count")
                        total_amount = inst.get("amount")
                        break
                if total_count is None:
                    total_count = sum((inst.get("count") or 0) for inst in instruments)
                    total_amount = sum((inst.get("amount") or 0.0) for inst in instruments)

                rows['State'].append(state)
                rows['Year'].append(year)
                rows['Quarter'].append(int(qfile.stem))
                rows['Transaction_type'].append(name)
                rows['Transaction_count'].append(total_count)
                rows['Transaction_amount'].append(total_amount)

df_aggregated_transaction = pd.DataFrame(rows)

# ---- Save CSV (make sure the directory exists) ----
out_dir = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out")
out_dir.mkdir(parents=True, exist_ok=True)
out_csv = out_dir / "aggregated_transactions.csv"

if df_aggregated_transaction.empty:
    print("[INFO] No rows parsed — check the BASE path or JSON structure.")
else:
    df_aggregated_transaction.sort_values(
        ['State', 'Year', 'Quarter', 'Transaction_type'],
        inplace=True, ignore_index=True
    )
    df_aggregated_transaction.to_csv(out_csv, index=False)
    print(f"[OK] {len(df_aggregated_transaction):,} rows written to: {out_csv}")


[OK] 5,034 rows written to: /Users/kishore_kumar/PhonePe/pulse/csv_out/aggregated_transactions.csv


In [6]:
#Aggregated Insurance
import os, json
import pandas as pd
from pathlib import Path

BASE_INS = Path("/Users/kishore_kumar/PhonePe/pulse/data/aggregated/insurance/country/india/state")
OUT_DIR = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

rows_ins = {
    'State': [], 'Year': [], 'Quarter': [],
    'Insurance_type': [], 'Insurance_count': [], 'Insurance_amount': []
}

if not BASE_INS.exists():
    raise FileNotFoundError(f"Base path not found: {BASE_INS}")

for state_dir in sorted([d for d in BASE_INS.iterdir() if d.is_dir()]):
    state = state_dir.name
    for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()]):
        year = int(year_dir.name)
        for qfile in sorted([f for f in year_dir.iterdir() if f.suffix == ".json"], key=lambda p: int(p.stem)):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)

            txn_list = (js or {}).get("data", {}).get("transactionData", []) or []
            for item in txn_list:
                name = item.get("name")
                instruments = item.get("paymentInstruments", []) or []

                # Prefer TOTAL; else sum
                total_count, total_amount = None, None
                for inst in instruments:
                    if inst.get("type") == "TOTAL":
                        total_count = inst.get("count")
                        total_amount = inst.get("amount")
                        break
                if total_count is None:
                    total_count = sum((inst.get("count") or 0) for inst in instruments)
                    total_amount = sum((inst.get("amount") or 0.0) for inst in instruments)

                rows_ins['State'].append(state)
                rows_ins['Year'].append(year)
                rows_ins['Quarter'].append(int(qfile.stem))
                rows_ins['Insurance_type'].append(name)
                rows_ins['Insurance_count'].append(total_count)
                rows_ins['Insurance_amount'].append(total_amount)

df_insurance = pd.DataFrame(rows_ins).sort_values(['State','Year','Quarter','Insurance_type'], ignore_index=True)
out_ins = OUT_DIR / "aggregated_insurance.csv"
df_insurance.to_csv(out_ins, index=False)
print(f"[OK] {len(df_insurance):,} rows -> {out_ins}")


[OK] 682 rows -> /Users/kishore_kumar/PhonePe/pulse/csv_out/aggregated_insurance.csv


In [7]:
#Aggregated User
import os, json
import pandas as pd
from pathlib import Path

BASE_USER = Path("/Users/kishore_kumar/PhonePe/pulse/data/aggregated/user/country/india/state")
OUT_DIR = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

rows_user_tot = {'State':[], 'Year':[], 'Quarter':[], 'Registered_users':[], 'App_opens':[]}
rows_user_dev = {'State':[], 'Year':[], 'Quarter':[], 'Brand':[], 'Brand_count':[], 'Brand_pct':[]}

if not BASE_USER.exists():
    raise FileNotFoundError(f"Base path not found: {BASE_USER}")

for state_dir in sorted([d for d in BASE_USER.iterdir() if d.is_dir()]):
    state = state_dir.name
    for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()]):
        year = int(year_dir.name)
        for qfile in sorted([f for f in year_dir.iterdir() if f.suffix == ".json"], key=lambda p: int(p.stem)):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)

            data = (js or {}).get("data", {}) or {}
            agg = data.get("aggregated", {}) or {}
            users_by_device = data.get("usersByDevice", []) or []

            # totals
            rows_user_tot['State'].append(state)
            rows_user_tot['Year'].append(year)
            rows_user_tot['Quarter'].append(int(qfile.stem))
            rows_user_tot['Registered_users'].append(agg.get("registeredUsers"))
            rows_user_tot['App_opens'].append(agg.get("appOpens"))

            # devices
            for d in users_by_device:
                rows_user_dev['State'].append(state)
                rows_user_dev['Year'].append(year)
                rows_user_dev['Quarter'].append(int(qfile.stem))
                rows_user_dev['Brand'].append(d.get("brand"))
                rows_user_dev['Brand_count'].append(d.get("count"))
                rows_user_dev['Brand_pct'].append(d.get("percentage"))

df_user_tot = pd.DataFrame(rows_user_tot).sort_values(['State','Year','Quarter'], ignore_index=True)
df_user_dev = pd.DataFrame(rows_user_dev).sort_values(['State','Year','Quarter','Brand'], ignore_index=True)

out_user_tot = OUT_DIR / "aggregated_users.csv"
out_user_dev = OUT_DIR / "aggregated_users_by_device.csv"
df_user_tot.to_csv(out_user_tot, index=False)
df_user_dev.to_csv(out_user_dev, index=False)
print(f"[OK] {len(df_user_tot):,} rows -> {out_user_tot}")
print(f"[OK] {len(df_user_dev):,} rows -> {out_user_dev}")


[OK] 1,008 rows -> /Users/kishore_kumar/PhonePe/pulse/csv_out/aggregated_users.csv
[OK] 6,732 rows -> /Users/kishore_kumar/PhonePe/pulse/csv_out/aggregated_users_by_device.csv


In [9]:
#map transaction
import json
import pandas as pd
from pathlib import Path

# Per README: data/map/transaction/hover/country/india/state/<state>/<year>/<quarter>.json
# (district-level details per state-year-quarter)
BASE = Path("/Users/kishore_kumar/PhonePe/pulse/data/map/transaction/hover/country/india/state")
OUT = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out"); OUT.mkdir(parents=True, exist_ok=True)

if not BASE.exists():
    raise FileNotFoundError(f"Path not found (check the 'hover' segment): {BASE}")

rows = {'State':[], 'Year':[], 'Quarter':[], 'Name':[], 'Metric_type':[], 'Count':[], 'Amount':[]}

for state_dir in sorted([d for d in BASE.iterdir() if d.is_dir()]):
    state = state_dir.name
    for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()]):
        year = int(year_dir.name)
        for qfile in sorted([f for f in year_dir.iterdir() if f.suffix==".json"], key=lambda p: int(p.stem)):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)
            hover_list = (js or {}).get("data", {}).get("hoverDataList", []) or []
            for h in hover_list:
                name = h.get("name")
                for m in (h.get("metric") or []):
                    rows['State'].append(state)
                    rows['Year'].append(year)
                    rows['Quarter'].append(int(qfile.stem))
                    rows['Name'].append(name)
                    rows['Metric_type'].append(m.get("type"))
                    rows['Count'].append(m.get("count"))
                    rows['Amount'].append(m.get("amount"))

df_map_txn = pd.DataFrame(rows).sort_values(['State','Year','Quarter','Name','Metric_type'], ignore_index=True)
out_csv = OUT / "map_transactions.csv"
df_map_txn.to_csv(out_csv, index=False)
print(f"[OK] {len(df_map_txn):,} rows -> {out_csv}")


[OK] 20,604 rows -> /Users/kishore_kumar/PhonePe/pulse/csv_out/map_transactions.csv


In [10]:
#map Insurance
import json
import pandas as pd
from pathlib import Path

# Per README: data/map/insurance/hover/country/india/state/<state>/<year>/<quarter>.json
BASE = Path("/Users/kishore_kumar/PhonePe/pulse/data/map/insurance/hover/country/india/state")
OUT = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out"); OUT.mkdir(parents=True, exist_ok=True)

if not BASE.exists():
    raise FileNotFoundError(f"Path not found (check the 'hover' segment): {BASE}")

rows = {'State':[], 'Year':[], 'Quarter':[], 'Name':[], 'Metric_type':[], 'Count':[], 'Amount':[]}

for state_dir in sorted([d for d in BASE.iterdir() if d.is_dir()]):
    state = state_dir.name
    for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()]):
        year = int(year_dir.name)
        for qfile in sorted([f for f in year_dir.iterdir() if f.suffix==".json"], key=lambda p: int(p.stem)):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)
            hover_list = (js or {}).get("data", {}).get("hoverDataList", []) or []
            for h in hover_list:
                name = h.get("name")
                for m in (h.get("metric") or []):
                    rows['State'].append(state)
                    rows['Year'].append(year)
                    rows['Quarter'].append(int(qfile.stem))
                    rows['Name'].append(name)
                    rows['Metric_type'].append(m.get("type"))
                    rows['Count'].append(m.get("count"))
                    rows['Amount'].append(m.get("amount"))

df_map_ins = pd.DataFrame(rows).sort_values(['State','Year','Quarter','Name','Metric_type'], ignore_index=True)
out_csv = OUT / "map_insurance.csv"
df_map_ins.to_csv(out_csv, index=False)
print(f"[OK] {len(df_map_ins):,} rows -> {out_csv}")


[OK] 13,876 rows -> /Users/kishore_kumar/PhonePe/pulse/csv_out/map_insurance.csv


In [11]:
#map user
import json
import pandas as pd
from pathlib import Path

# Per README: data/map/user/hover/country/india/state/<state>/<year>/<quarter>.json
BASE = Path("/Users/kishore_kumar/PhonePe/pulse/data/map/user/hover/country/india/state")
OUT = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out"); OUT.mkdir(parents=True, exist_ok=True)

if not BASE.exists():
    raise FileNotFoundError(f"Path not found (check the 'hover' segment): {BASE}")

rows = {'State':[], 'Year':[], 'Quarter':[], 'Name':[], 'Registered_users':[], 'App_opens':[]}

for state_dir in sorted([d for d in BASE.iterdir() if d.is_dir()]):
    state = state_dir.name
    for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()]):
        year = int(year_dir.name)
        for qfile in sorted([f for f in year_dir.iterdir() if f.suffix==".json"], key=lambda p: int(p.stem)):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)
            hover = (js or {}).get("data", {}).get("hoverData", {}) or {}
            for name, v in hover.items():
                rows['State'].append(state)
                rows['Year'].append(year)
                rows['Quarter'].append(int(qfile.stem))
                rows['Name'].append(name)
                rows['Registered_users'].append((v or {}).get("registeredUsers"))
                rows['App_opens'].append((v or {}).get("appOpens"))

df_map_user = pd.DataFrame(rows).sort_values(['State','Year','Quarter','Name'], ignore_index=True)
out_csv = OUT / "map_users.csv"
df_map_user.to_csv(out_csv, index=False)
print(f"[OK] {len(df_map_user):,} rows -> {out_csv}")


[OK] 20,608 rows -> /Users/kishore_kumar/PhonePe/pulse/csv_out/map_users.csv


In [12]:
import json
from pathlib import Path
import pandas as pd

# ---- Adjust these two paths only ----
ROOT = Path("/Users/kishore_kumar/PhonePe/pulse/data")  # repo's data/ path
OUT  = Path("/Users/kishore_kumar/PhonePe/pulse")

OUT.mkdir(parents=True, exist_ok=True)

# Helpers
def _safe_int(s): 
    try: return int(s)
    except: return None

def _iter_quarters(dir_path: Path):
    """Yield quarter files sorted by integer stem."""
    qfiles = [f for f in dir_path.iterdir() if f.suffix == ".json"]
    return sorted(qfiles, key=lambda p: _safe_int(p.stem) if p.stem.isdigit() else 999)

# ---------- STATE LEVEL ----------
def map_transaction_state():
    base = ROOT / "map" / "transaction" / "hover" / "country" / "india" / "state"
    if not base.exists(): 
        raise FileNotFoundError(f"Not found: {base}")

    rows = {'State':[], 'Year':[], 'Quarter':[], 'Name':[], 'Metric_type':[], 'Count':[], 'Amount':[]}

    for state_dir in sorted([d for d in base.iterdir() if d.is_dir()]):
        state = state_dir.name
        for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()], key=lambda p: int(p.name)):
            year = int(year_dir.name)
            for qfile in _iter_quarters(year_dir):
                with qfile.open("r", encoding="utf-8") as f:
                    js = json.load(f)
                hover_list = (js or {}).get("data", {}).get("hoverDataList", []) or []
                for h in hover_list:
                    name = h.get("name")
                    for m in (h.get("metric") or []):
                        rows['State'].append(state)
                        rows['Year'].append(year)
                        rows['Quarter'].append(int(qfile.stem))
                        rows['Name'].append(name)
                        rows['Metric_type'].append(m.get("type"))
                        rows['Count'].append(m.get("count"))
                        rows['Amount'].append(m.get("amount"))

    df = pd.DataFrame(rows).sort_values(['State','Year','Quarter','Name','Metric_type'], ignore_index=True)
    out = OUT / "map_transactions_state.csv"
    df.to_csv(out, index=False)
    print(f"[OK] {len(df):,} rows -> {out}")

def map_insurance_state():
    base = ROOT / "map" / "insurance" / "hover" / "country" / "india" / "state"
    if not base.exists(): 
        raise FileNotFoundError(f"Not found: {base}")

    rows = {'State':[], 'Year':[], 'Quarter':[], 'Name':[], 'Metric_type':[], 'Count':[], 'Amount':[]}

    for state_dir in sorted([d for d in base.iterdir() if d.is_dir()]):
        state = state_dir.name
        for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()], key=lambda p: int(p.name)):
            year = int(year_dir.name)
            for qfile in _iter_quarters(year_dir):
                with qfile.open("r", encoding="utf-8") as f:
                    js = json.load(f)
                hover_list = (js or {}).get("data", {}).get("hoverDataList", []) or []
                for h in hover_list:
                    name = h.get("name")
                    for m in (h.get("metric") or []):
                        rows['State'].append(state)
                        rows['Year'].append(year)
                        rows['Quarter'].append(int(qfile.stem))
                        rows['Name'].append(name)
                        rows['Metric_type'].append(m.get("type"))
                        rows['Count'].append(m.get("count"))
                        rows['Amount'].append(m.get("amount"))

    df = pd.DataFrame(rows).sort_values(['State','Year','Quarter','Name','Metric_type'], ignore_index=True)
    out = OUT / "map_insurance_state.csv"
    df.to_csv(out, index=False)
    print(f"[OK] {len(df):,} rows -> {out}")

def map_user_state():
    base = ROOT / "map" / "user" / "hover" / "country" / "india" / "state"
    if not base.exists(): 
        raise FileNotFoundError(f"Not found: {base}")

    rows = {'State':[], 'Year':[], 'Quarter':[], 'Name':[], 'Registered_users':[], 'App_opens':[]}

    for state_dir in sorted([d for d in base.iterdir() if d.is_dir()]):
        state = state_dir.name
        for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()], key=lambda p: int(p.name)):
            year = int(year_dir.name)
            for qfile in _iter_quarters(year_dir):
                with qfile.open("r", encoding="utf-8") as f:
                    js = json.load(f)
                hover = (js or {}).get("data", {}).get("hoverData", {}) or {}
                for name, v in hover.items():
                    rows['State'].append(state)
                    rows['Year'].append(year)
                    rows['Quarter'].append(int(qfile.stem))
                    rows['Name'].append(name)
                    rows['Registered_users'].append((v or {}).get("registeredUsers"))
                    rows['App_opens'].append((v or {}).get("appOpens"))

    df = pd.DataFrame(rows).sort_values(['State','Year','Quarter','Name'], ignore_index=True)
    out = OUT / "map_users_state.csv"
    df.to_csv(out, index=False)
    print(f"[OK] {len(df):,} rows -> {out}")

# ---------- COUNTRY LEVEL ----------
def map_transaction_country():
    base = ROOT / "map" / "transaction" / "hover" / "country" / "india"
    if not base.exists(): 
        raise FileNotFoundError(f"Not found: {base}")

    rows = {'Year':[], 'Quarter':[], 'Name':[], 'Metric_type':[], 'Count':[], 'Amount':[]}

    # layout: .../india/<year>/<quarter>.json
    for year_dir in sorted([d for d in base.iterdir() if d.is_dir() and d.name != "state"], key=lambda p: int(p.name)):
        year = int(year_dir.name)
        for qfile in _iter_quarters(year_dir):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)
            hover_list = (js or {}).get("data", {}).get("hoverDataList", []) or []
            for h in hover_list:
                name = h.get("name")
                for m in (h.get("metric") or []):
                    rows['Year'].append(year)
                    rows['Quarter'].append(int(qfile.stem))
                    rows['Name'].append(name)
                    rows['Metric_type'].append(m.get("type"))
                    rows['Count'].append(m.get("count"))
                    rows['Amount'].append(m.get("amount"))

    df = pd.DataFrame(rows).sort_values(['Year','Quarter','Name','Metric_type'], ignore_index=True)
    out = OUT / "map_transactions_country.csv"
    df.to_csv(out, index=False)
    print(f"[OK] {len(df):,} rows -> {out}")

def map_insurance_country():
    base = ROOT / "map" / "insurance" / "hover" / "country" / "india"
    if not base.exists(): 
        raise FileNotFoundError(f"Not found: {base}")

    rows = {'Year':[], 'Quarter':[], 'Name':[], 'Metric_type':[], 'Count':[], 'Amount':[]}

    for year_dir in sorted([d for d in base.iterdir() if d.is_dir() and d.name != "state"], key=lambda p: int(p.name)):
        year = int(year_dir.name)
        for qfile in _iter_quarters(year_dir):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)
            hover_list = (js or {}).get("data", {}).get("hoverDataList", []) or []
            for h in hover_list:
                name = h.get("name")
                for m in (h.get("metric") or []):
                    rows['Year'].append(year)
                    rows['Quarter'].append(int(qfile.stem))
                    rows['Name'].append(name)
                    rows['Metric_type'].append(m.get("type"))
                    rows['Count'].append(m.get("count"))
                    rows['Amount'].append(m.get("amount"))

    df = pd.DataFrame(rows).sort_values(['Year','Quarter','Name','Metric_type'], ignore_index=True)
    out = OUT / "map_insurance_country.csv"
    df.to_csv(out, index=False)
    print(f"[OK] {len(df):,} rows -> {out}")

def map_user_country():
    base = ROOT / "map" / "user" / "hover" / "country" / "india"
    if not base.exists(): 
        raise FileNotFoundError(f"Not found: {base}")

    rows = {'Year':[], 'Quarter':[], 'Name':[], 'Registered_users':[], 'App_opens':[]}

    for year_dir in sorted([d for d in base.iterdir() if d.is_dir() and d.name != "state"], key=lambda p: int(p.name)):
        year = int(year_dir.name)
        for qfile in _iter_quarters(year_dir):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)
            hover = (js or {}).get("data", {}).get("hoverData", {}) or {}
            for name, v in hover.items():
                rows['Year'].append(year)
                rows['Quarter'].append(int(qfile.stem))
                rows['Name'].append(name)
                rows['Registered_users'].append((v or {}).get("registeredUsers"))
                rows['App_opens'].append((v or {}).get("appOpens"))

    df = pd.DataFrame(rows).sort_values(['Year','Quarter','Name'], ignore_index=True)
    out = OUT / "map_users_country.csv"
    df.to_csv(out, index=False)
    print(f"[OK] {len(df):,} rows -> {out}")

# Run all
if __name__ == "__main__":
    map_transaction_state()
    map_insurance_state()
    map_user_state()
    map_transaction_country()
    map_insurance_country()
    map_user_country()


[OK] 20,604 rows -> /Users/kishore_kumar/PhonePe/pulse/map_transactions_state.csv
[OK] 13,876 rows -> /Users/kishore_kumar/PhonePe/pulse/map_insurance_state.csv
[OK] 20,608 rows -> /Users/kishore_kumar/PhonePe/pulse/map_users_state.csv
[OK] 1,008 rows -> /Users/kishore_kumar/PhonePe/pulse/map_transactions_country.csv
[OK] 682 rows -> /Users/kishore_kumar/PhonePe/pulse/map_insurance_country.csv
[OK] 1,008 rows -> /Users/kishore_kumar/PhonePe/pulse/map_users_country.csv


In [15]:
import json
from pathlib import Path
import pandas as pd

# ======= Set your paths =======
ROOT = Path("/Users/kishore_kumar/PhonePe/pulse/data")   # path to repo's data/ folder
OUT  = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out")
OUT.mkdir(parents=True, exist_ok=True)

def _safe_int(s):
    try: return int(s)
    except: return None

def _iter_quarters(dir_path: Path):
    qfiles = [f for f in dir_path.iterdir() if f.suffix == ".json"]
    return sorted(qfiles, key=lambda p: _safe_int(p.stem) if p.stem.isdigit() else 999)

rows = []

# ---------- STATE LEVEL ONLY ----------
base_state = ROOT / "top" / "insurance" / "country" / "india" / "state"
if not base_state.exists():
    raise FileNotFoundError(f"Not found: {base_state}")

for state_dir in sorted([d for d in base_state.iterdir() if d.is_dir()]):
    state = state_dir.name
    for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()], key=lambda p: int(p.name)):
        year = int(year_dir.name)
        for qfile in _iter_quarters(year_dir):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)
            data = (js or {}).get("data", {}) or {}

            # DISTRICTS
            for e in data.get("districts", []) or []:
                m = e.get("metric", {}) or {}
                rows.append({
                    "State": state,
                    "Entity_Level": "district",
                    "Entity": e.get("entityName"),
                    "Metric_type": m.get("type"),
                    "Count": m.get("count"),
                    "Amount": m.get("amount"),
                    "Year": year,
                    "Quarter": int(qfile.stem),
                })

            # PINCODES
            for e in data.get("pincodes", []) or []:
                m = e.get("metric", {}) or {}
                rows.append({
                    "State": state,
                    "Entity_Level": "pincode",
                    "Entity": e.get("entityName"),
                    "Metric_type": m.get("type"),
                    "Count": m.get("count"),
                    "Amount": m.get("amount"),
                    "Year": year,
                    "Quarter": int(qfile.stem),
                })

# ---------- Build & write ----------
df = pd.DataFrame(rows)
if df.empty:
    print("[INFO] No rows parsed — check paths and repo contents.")
else:
    df.sort_values(["State","Year","Quarter","Entity_Level","Entity"], inplace=True, ignore_index=True)
    out_csv = OUT / "top_insurance_state_combined.csv"
    df.to_csv(out_csv, index=False)
    print(f"[OK] {len(df):,} rows -> {out_csv}")


[OK] 12,276 rows -> /Users/kishore_kumar/PhonePe/pulse/csv_out/top_insurance_state_combined.csv


In [16]:
import json
from pathlib import Path
import pandas as pd

# ======= Set your paths =======
ROOT = Path("/Users/kishore_kumar/PhonePe/pulse/data")   # path to repo's data/ folder
OUT  = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out")
OUT.mkdir(parents=True, exist_ok=True)

def _safe_int(s):
    try: return int(s)
    except: return None

def _iter_quarters(dir_path: Path):
    qfiles = [f for f in dir_path.iterdir() if f.suffix == ".json"]
    return sorted(qfiles, key=lambda p: _safe_int(p.stem) if p.stem.isdigit() else 999)

# ---------- Top → Transactions (STATE)  ----------
def build_top_transactions_state_combined():
    base_state = ROOT / "top" / "transaction" / "country" / "india" / "state"
    if not base_state.exists():
        raise FileNotFoundError(f"Not found: {base_state}")

    rows = []
    for state_dir in sorted([d for d in base_state.iterdir() if d.is_dir()]):
        state = state_dir.name
        for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()], key=lambda p: int(p.name)):
            year = int(year_dir.name)
            for qfile in _iter_quarters(year_dir):
                with qfile.open("r", encoding="utf-8") as f:
                    js = json.load(f)
                data = (js or {}).get("data", {}) or {}

                # DISTRICTS
                for e in data.get("districts", []) or []:
                    m = e.get("metric", {}) or {}
                    rows.append({
                        "State": state,
                        "Entity_Level": "district",
                        "Entity": e.get("entityName"),
                        "Metric_type": m.get("type"),
                        "Count": m.get("count"),
                        "Amount": m.get("amount"),
                        "Year": year,
                        "Quarter": int(qfile.stem),
                    })

                # PINCODES
                for e in data.get("pincodes", []) or []:
                    m = e.get("metric", {}) or {}
                    rows.append({
                        "State": state,
                        "Entity_Level": "pincode",
                        "Entity": e.get("entityName"),
                        "Metric_type": m.get("type"),
                        "Count": m.get("count"),
                        "Amount": m.get("amount"),
                        "Year": year,
                        "Quarter": int(qfile.stem),
                    })

    df = pd.DataFrame(rows)
    if df.empty:
        print("[INFO] Transactions (state) — no rows parsed. Check paths.")
        return
    df.sort_values(["State","Year","Quarter","Entity_Level","Entity"], inplace=True, ignore_index=True)
    out_csv = OUT / "top_transactions_state_combined.csv"
    df.to_csv(out_csv, index=False)
    print(f"[OK] {len(df):,} rows -> {out_csv}")


# ---------- Run ----------
if __name__ == "__main__":
    build_top_transactions_state_combined()
   


[OK] 18,295 rows -> /Users/kishore_kumar/PhonePe/pulse/csv_out/top_transactions_state_combined.csv


In [17]:
import json
from pathlib import Path
import pandas as pd

# ======= Set your paths =======
ROOT = Path("/Users/kishore_kumar/PhonePe/pulse/data")   # path to repo's data/ folder
OUT  = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out")
OUT.mkdir(parents=True, exist_ok=True)

def _safe_int(s):
    try: return int(s)
    except: return None

def _iter_quarters(dir_path: Path):
    qfiles = [f for f in dir_path.iterdir() if f.suffix == ".json"]
    return sorted(qfiles, key=lambda p: _safe_int(p.stem) if p.stem.isdigit() else 999)
# ---------- Top → Users (STATE) ----------
def build_top_users_state_combined():
    base_state = ROOT / "top" / "user" / "country" / "india" / "state"
    if not base_state.exists():
        raise FileNotFoundError(f"Not found: {base_state}")

    rows = []
    for state_dir in sorted([d for d in base_state.iterdir() if d.is_dir()]):
        state = state_dir.name
        for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()], key=lambda p: int(p.name)):
            year = int(year_dir.name)
            for qfile in _iter_quarters(year_dir):
                with qfile.open("r", encoding="utf-8") as f:
                    js = json.load(f)
                data = (js or {}).get("data", {}) or {}

                # DISTRICTS
                for e in data.get("districts", []) or []:
                    rows.append({
                        "State": state,
                        "Entity_Level": "district",
                        "Entity": e.get("name"),
                        "Registered_users": e.get("registeredUsers"),
                        "Year": year,
                        "Quarter": int(qfile.stem),
                    })

                # PINCODES
                for e in data.get("pincodes", []) or []:
                    rows.append({
                        "State": state,
                        "Entity_Level": "pincode",
                        "Entity": e.get("name"),
                        "Registered_users": e.get("registeredUsers"),
                        "Year": year,
                        "Quarter": int(qfile.stem),
                    })

    df = pd.DataFrame(rows)
    if df.empty:
        print("[INFO] Users (state) — no rows parsed. Check paths.")
        return
    df.sort_values(["State","Year","Quarter","Entity_Level","Entity"], inplace=True, ignore_index=True)
    out_csv = OUT / "top_users_state_combined.csv"
    df.to_csv(out_csv, index=False)
    print(f"[OK] {len(df):,} rows -> {out_csv}")

# ---------- Run ----------
if __name__ == "__main__":
    build_top_users_state_combined()


[OK] 18,296 rows -> /Users/kishore_kumar/PhonePe/pulse/csv_out/top_users_state_combined.csv


In [18]:
import pandas as pd

# adjust the path to your combined CSV
df = pd.read_csv("/Users/kishore_kumar/PhonePe/pulse/csv_out/top_insurance_state_combined.csv")

print(df.head())


                       State Entity_Level         Entity Metric_type  Count  \
0  andaman-&-nicobar-islands     district       nicobars       TOTAL      3   
1  andaman-&-nicobar-islands     district  south andaman       TOTAL      3   
2  andaman-&-nicobar-islands      pincode         744101       TOTAL      1   
3  andaman-&-nicobar-islands      pincode         744104       TOTAL      2   
4  andaman-&-nicobar-islands      pincode         744301       TOTAL      3   

   Amount  Year  Quarter  
0   565.0  2020        2  
1   795.0  2020        2  
2   282.0  2020        2  
3   513.0  2020        2  
4   565.0  2020        2  


In [19]:
# filter by Entity_Level
df_districts = df[df["Entity_Level"] == "district"].copy()
df_pincodes  = df[df["Entity_Level"] == "pincode"].copy()

print("District rows:", len(df_districts))
print("Pincode rows:", len(df_pincodes))


District rows: 5608
Pincode rows: 6668


In [20]:
out_dir = "/Users/kishore_kumar/PhonePe/pulse/csv_out"

df_districts.to_csv(f"{out_dir}/top_insurance_state_districts.csv", index=False)
df_pincodes.to_csv(f"{out_dir}/top_insurance_state_pincodes.csv", index=False)

print("✅ Wrote two separate CSVs:")
print(f"{out_dir}/top_insurance_state_districts.csv")
print(f"{out_dir}/top_insurance_state_pincodes.csv")


✅ Wrote two separate CSVs:
/Users/kishore_kumar/PhonePe/pulse/csv_out/top_insurance_state_districts.csv
/Users/kishore_kumar/PhonePe/pulse/csv_out/top_insurance_state_pincodes.csv


In [2]:
import pandas as pd

# adjust the path to your combined CSV
df = pd.read_csv("/Users/kishore_kumar/PhonePe/pulse/csv_out/top_transactions_state_combined.csv")

print(df.head())

                       State Entity_Level                    Entity  \
0  andaman-&-nicobar-islands     district                  nicobars   
1  andaman-&-nicobar-islands     district  north and middle andaman   
2  andaman-&-nicobar-islands     district             south andaman   
3  andaman-&-nicobar-islands      pincode                    744101   
4  andaman-&-nicobar-islands      pincode                    744102   

  Metric_type  Count        Amount  Year  Quarter  
0       TOTAL    528  1.139849e+06  2018        1  
1       TOTAL    442  9.316631e+05  2018        1  
2       TOTAL   5688  1.256025e+07  2018        1  
3       TOTAL   1622  2.769298e+06  2018        1  
4       TOTAL    969  3.519060e+06  2018        1  


In [3]:
# filter by Entity_Level
df_districts = df[df["Entity_Level"] == "district"].copy()
df_pincodes  = df[df["Entity_Level"] == "pincode"].copy()

print("District rows:", len(df_districts))
print("Pincode rows:", len(df_pincodes))

out_dir = "/Users/kishore_kumar/PhonePe/pulse/csv_out"

df_districts.to_csv(f"{out_dir}/top_transaction_state_districts.csv", index=False)
df_pincodes.to_csv(f"{out_dir}/top_transaction_state_pincodes.csv", index=False)

print("✅ Wrote two separate CSVs:")
print(f"{out_dir}/top_transaction_state_districts.csv")
print(f"{out_dir}/top_transaction_state_pincodes.csv")


District rows: 8296
Pincode rows: 9999
✅ Wrote two separate CSVs:
/Users/kishore_kumar/PhonePe/pulse/csv_out/top_transaction_state_districts.csv
/Users/kishore_kumar/PhonePe/pulse/csv_out/top_transaction_state_pincodes.csv


import json
from pathlib import Path
import pandas as pd

# ======= Configure these two paths =======
ROOT = Path("/Users/kishore_kumar/PhonePe/pulse/data")  # path to the repo's data/ folder
OUT  = Path("/Users/kishore_kumar/PhonePe/pulse/csv_out")
OUT.mkdir(parents=True, exist_ok=True)

def _safe_int(s):
    try: return int(s)
    except: return None

def _iter_quarters(dir_path: Path):
    qfiles = [f for f in dir_path.iterdir() if f.suffix == ".json"]
    return sorted(qfiles, key=lambda p: _safe_int(p.stem) if p.stem.isdigit() else 999)

# ---------- COUNTRY LEVEL HELPERS ----------
def _top_country_common(base: Path, users_mode: bool):
    """
    Reads country-level top files at base=.../data/top/<type>/country/india
    Writes three CSVs per type: states, districts, pincodes
    """
    if not base.exists():
        raise FileNotFoundError(f"Not found: {base}")

    rows_states = []
    rows_districts = []
    rows_pincodes = []

    for year_dir in sorted([d for d in base.iterdir() if d.is_dir() and d.name != "state"], key=lambda p: int(p.name)):
        year = int(year_dir.name)
        for qfile in _iter_quarters(year_dir):
            with qfile.open("r", encoding="utf-8") as f:
                js = json.load(f)
            data = (js or {}).get("data", {}) or {}

            # STATES (country-level has states)
            for e in data.get("states", []) or []:
                if users_mode:
                    rows_states.append({
                        "Year": year, "Quarter": int(qfile.stem),
                        "Entity": e.get("name"),
                        "Registered_users": e.get("registeredUsers")
                    })
                else:
                    m = e.get("metric", {}) or {}
                    rows_states.append({
                        "Year": year, "Quarter": int(qfile.stem),
                        "Entity": e.get("entityName"),
                        "Metric_type": m.get("type"),
                        "Count": m.get("count"),
                        "Amount": m.get("amount"),
                    })

            # DISTRICTS
            for e in data.get("districts", []) or []:
                if users_mode:
                    rows_districts.append({
                        "Year": year, "Quarter": int(qfile.stem),
                        "Entity": e.get("name"),
                        "Registered_users": e.get("registeredUsers")
                    })
                else:
                    m = e.get("metric", {}) or {}
                    rows_districts.append({
                        "Year": year, "Quarter": int(qfile.stem),
                        "Entity": e.get("entityName"),
                        "Metric_type": m.get("type"),
                        "Count": m.get("count"),
                        "Amount": m.get("amount"),
                    })

            # PINCODES
            for e in data.get("pincodes", []) or []:
                if users_mode:
                    rows_pincodes.append({
                        "Year": year, "Quarter": int(qfile.stem),
                        "Entity": e.get("name"),
                        "Registered_users": e.get("registeredUsers")
                    })
                else:
                    m = e.get("metric", {}) or {}
                    rows_pincodes.append({
                        "Year": year, "Quarter": int(qfile.stem),
                        "Entity": e.get("entityName"),
                        "Metric_type": m.get("type"),
                        "Count": m.get("count"),
                        "Amount": m.get("amount"),
                    })

    return rows_states, rows_districts, rows_pincodes

# ---------- STATE LEVEL HELPERS ----------
def _top_state_common(base: Path, users_mode: bool):
    """
    Reads state-level top files at base=.../data/top/<type>/country/india/state
    Writes two CSVs per type: districts, pincodes (README: no 'states' at state-level)
    """
    if not base.exists():
        raise FileNotFoundError(f"Not found: {base}")

    rows_districts = []
    rows_pincodes = []

    for state_dir in sorted([d for d in base.iterdir() if d.is_dir()]):
        state = state_dir.name
        for year_dir in sorted([d for d in state_dir.iterdir() if d.is_dir()], key=lambda p: int(p.name)):
            year = int(year_dir.name)
            for qfile in _iter_quarters(year_dir):
                with qfile.open("r", encoding="utf-8") as f:
                    js = json.load(f)
                data = (js or {}).get("data", {}) or {}

                # DISTRICTS
                for e in data.get("districts", []) or []:
                    if users_mode:
                        rows_districts.append({
                            "State": state, "Year": year, "Quarter": int(qfile.stem),
                            "Entity": e.get("name"),
                            "Registered_users": e.get("registeredUsers")
                        })
                    else:
                        m = e.get("metric", {}) or {}
                        rows_districts.append({
                            "State": state, "Year": year, "Quarter": int(qfile.stem),
                            "Entity": e.get("entityName"),
                            "Metric_type": m.get("type"),
                            "Count": m.get("count"),
                            "Amount": m.get("amount"),
                        })

                # PINCODES
                for e in data.get("pincodes", []) or []:
                    if users_mode:
                        rows_pincodes.append({
                            "State": state, "Year": year, "Quarter": int(qfile.stem),
                            "Entity": e.get("name"),
                            "Registered_users": e.get("registeredUsers")
                        })
                    else:
                        m = e.get("metric", {}) or {}
                        rows_pincodes.append({
                            "State": state, "Year": year, "Quarter": int(qfile.stem),
                            "Entity": e.get("entityName"),
                            "Metric_type": m.get("type"),
                            "Count": m.get("count"),
                            "Amount": m.get("amount"),
                        })

    return rows_districts, rows_pincodes

# ---------- TRANSACTION ----------
def top_transaction_country():
    base = ROOT / "top" / "transaction" / "country" / "india"
    st, di, pi = _top_country_common(base, users_mode=False)
    pd.DataFrame(st).sort_values(["Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_transactions_states_country.csv", index=False)
    pd.DataFrame(di).sort_values(["Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_transactions_districts_country.csv", index=False)
    pd.DataFrame(pi).sort_values(["Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_transactions_pincodes_country.csv", index=False)

def top_transaction_state():
    base = ROOT / "top" / "transaction" / "country" / "india" / "state"
    di, pi = _top_state_common(base, users_mode=False)
    pd.DataFrame(di).sort_values(["State","Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_transactions_districts_state.csv", index=False)
    pd.DataFrame(pi).sort_values(["State","Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_transactions_pincodes_state.csv", index=False)

# ---------- INSURANCE ----------
def top_insurance_country():
    base = ROOT / "top" / "insurance" / "country" / "india"
    st, di, pi = _top_country_common(base, users_mode=False)
    pd.DataFrame(st).sort_values(["Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_insurance_states_country.csv", index=False)
    pd.DataFrame(di).sort_values(["Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_insurance_districts_country.csv", index=False)
    pd.DataFrame(pi).sort_values(["Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_insurance_pincodes_country.csv", index=False)

def top_insurance_state():
    base = ROOT / "top" / "insurance" / "country" / "india" / "state"
    di, pi = _top_state_common(base, users_mode=False)
    pd.DataFrame(di).sort_values(["State","Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_insurance_districts_state.csv", index=False)
    pd.DataFrame(pi).sort_values(["State","Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_insurance_pincodes_state.csv", index=False)

# ---------- USER ----------
def top_user_country():
    base = ROOT / "top" / "user" / "country" / "india"
    st, di, pi = _top_country_common(base, users_mode=True)
    pd.DataFrame(st).sort_values(["Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_users_states_country.csv", index=False)
    pd.DataFrame(di).sort_values(["Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_users_districts_country.csv", index=False)
    pd.DataFrame(pi).sort_values(["Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_users_pincodes_country.csv", index=False)

def top_user_state():
    base = ROOT / "top" / "user" / "country" / "india" / "state"
    di, pi = _top_state_common(base, users_mode=True)
    pd.DataFrame(di).sort_values(["State","Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_users_districts_state.csv", index=False)
    pd.DataFrame(pi).sort_values(["State","Year","Quarter","Entity"], ignore_index=True)\
        .to_csv(OUT / "top_users_pincodes_state.csv", index=False)

if __name__ == "__main__":
    top_transaction_country()
    top_transaction_state()
    top_insurance_country()
    top_insurance_state()
    top_user_country()
    top_user_state()
    print("[OK] Wrote Top CSVs to:", OUT)
